<a href="https://www.kaggle.com/code/samithsachidanandan/image-classification-cnn-in-pytorch?scriptVersionId=283706857" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import Libraries 

In [1]:
import numpy as np 
from PIL import Image

import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms



### Converting an image to a PyTorch tensor

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    
])

### Loading the Dataset

In [3]:
   import os
   data_dir = '/kaggle/working/data'
   os.makedirs(data_dir, exist_ok=True)

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:03<00:00, 55.5MB/s]


In [5]:
image, label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [8]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12,24,5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)   
        self.fc2 = nn.Linear(120, 84 )
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x  = F.relu(self.fc2(x)) 
        x = self.fc3(x) 
        return x  

In [9]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001, momentum=0.9)

In [10]:
for epoch in range(50):
    print(f'Training epoch {epoch} ...')


    running_loss = 0.0 

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len (train_loader) : .4f}' )

Training epoch 0 ...
Loss:  2.2171
Training epoch 1 ...
Loss:  1.7426
Training epoch 2 ...
Loss:  1.5095
Training epoch 3 ...
Loss:  1.3944
Training epoch 4 ...
Loss:  1.2974
Training epoch 5 ...
Loss:  1.2139
Training epoch 6 ...
Loss:  1.1469
Training epoch 7 ...
Loss:  1.0863
Training epoch 8 ...
Loss:  1.0360
Training epoch 9 ...
Loss:  0.9916
Training epoch 10 ...
Loss:  0.9504
Training epoch 11 ...
Loss:  0.9119
Training epoch 12 ...
Loss:  0.8738
Training epoch 13 ...
Loss:  0.8387
Training epoch 14 ...
Loss:  0.8107
Training epoch 15 ...
Loss:  0.7765
Training epoch 16 ...
Loss:  0.7470
Training epoch 17 ...
Loss:  0.7229
Training epoch 18 ...
Loss:  0.6958
Training epoch 19 ...
Loss:  0.6711
Training epoch 20 ...
Loss:  0.6440
Training epoch 21 ...
Loss:  0.6216
Training epoch 22 ...
Loss:  0.5997
Training epoch 23 ...
Loss:  0.5795
Training epoch 24 ...
Loss:  0.5505
Training epoch 25 ...
Loss:  0.5323
Training epoch 26 ...
Loss:  0.5105
Training epoch 27 ...
Loss:  0.4888
Tr

In [11]:
torch.save(net.state_dict(), 'trained_net.pth')

In [12]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [13]:
correct = 0 
total = 0 
net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 66.05%
